In [4]:
from keras import Input, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input

TRAIN_DIR = r"D:\NUS_TERM2_CA2\Train"
TEST_DIR = r"D:\NUS_TERM2_CA2\Validation"

HEIGHT = 300
WIDTH = 300
BATCH_SIZE = 8

datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=90,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=1./255)

datagen2 = ImageDataGenerator(
    rescale=1./255)


train_generator = datagen.flow_from_directory(TRAIN_DIR,
                                                    target_size=(HEIGHT, WIDTH),
                                                    batch_size=BATCH_SIZE)

validation_generator = datagen2.flow_from_directory(TEST_DIR,
                                                    target_size=(HEIGHT, WIDTH),
                                                    batch_size=BATCH_SIZE)


class_list = ["food", "landmark", "people"]



Found 2424 images belonging to 3 classes.
Found 720 images belonging to 3 classes.


In [25]:
from keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, BatchNormalization, MaxPooling2D
from keras.regularizers import l2
from keras.optimizers import SGD, Adam

seed = 29
np.random.seed(seed)

inputs = Input((HEIGHT, WIDTH, 3))

y = Conv2D(16, kernel_size=2, padding='same', kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(inputs)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D()(y)
y = Dropout(0.3)(y)

y = Conv2D(32, kernel_size=2, padding='same', kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D()(y)
y = Dropout(0.3)(y)

y = Conv2D(64, kernel_size=2, padding='same', kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D()(y)
y = Dropout(0.3)(y)

y = Conv2D(128, kernel_size=2, padding='same', kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(y)
y = BatchNormalization()(y)
y = Activation('relu')(y)
y = MaxPooling2D()(y)
y = Dropout(0.3)(y)

y = Flatten()(y)

y = Dense(len(class_list), activation='softmax')(y) 

model = Model(inputs=inputs, outputs=y)

opt = Adam(lr=0.001)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])

model.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 300, 300, 3)       0         
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 300, 300, 16)      208       
_________________________________________________________________
batch_normalization_75 (Batc (None, 300, 300, 16)      64        
_________________________________________________________________
activation_75 (Activation)   (None, 300, 300, 16)      0         
_________________________________________________________________
max_pooling2d_75 (MaxPooling (None, 150, 150, 16)      0         
_________________________________________________________________
dropout_84 (Dropout)         (None, 150, 150, 16)      0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 150, 150, 32)      208

In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler
import matplotlib.pyplot as plt
from PIL import Image

NUM_EPOCHS = 200
BATCH_SIZE = 64
num_train_images = 2424
num_test_images = 720

def lrSchedule(epoch):
    lr = 1e-2

    if epoch > 160:
        lr *= 0.5e-3

    elif epoch > 120:
        lr *= 1e-3

    elif epoch > 80:
        lr *= 1e-2

    elif epoch > 10:
        lr *= 1e-1

    print('Learning rate: ', lr)

    return lr


LRScheduler = LearningRateScheduler(lrSchedule)

modelname = 'normal_DL'
filepath = modelname + '.hdf5'

checkpoint = ModelCheckpoint(filepath,
                            monitor='val_acc',
                            verbose=0,
                            save_best_only=True,
                            mode='max')
csv_logger = CSVLogger(modelname + '.csv')

callbacks_list = [checkpoint, csv_logger, LRScheduler]

model.fit_generator(generator=train_generator,
                    validation_data=validation_generator,
                    epochs=NUM_EPOCHS,
                    workers=12, 
                    shuffle=True,
                    validation_steps=num_test_images // BATCH_SIZE,
                    steps_per_epoch=num_train_images // BATCH_SIZE, 
                    callbacks=callbacks_list)

Epoch 1/200


FailedPreconditionError: 2 root error(s) found.
  (0) Failed precondition: Attempting to use uninitialized value Adam_14/lr
	 [[node Adam_14/lr (defined at c:\users\guofe\workspace\nus_is_pr\venv\lib\site-packages\keras\backend\tensorflow_backend.py:422) ]]
	 [[Adam_14/lr/_5]]
  (1) Failed precondition: Attempting to use uninitialized value Adam_14/lr
	 [[node Adam_14/lr (defined at c:\users\guofe\workspace\nus_is_pr\venv\lib\site-packages\keras\backend\tensorflow_backend.py:422) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'Adam_14/lr':
  File "C:\Users\guofe\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\guofe\AppData\Local\Programs\Python\Python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\guofe\workspace\nus_is_pr\venv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\

In [ ]:
import matplotlib.pyplot as plt
# loss_values = history_dict['loss']
# val_loss_values = history_dict['val_loss']
plt.plot(history.history['acc'][7:])
plt.plot(history.history['val_acc'][7:])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'][10:])
plt.plot(history.history['val_loss'][10:])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict_generator(validation_generator, num_test_images//8, workers=12, verbose=1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = class_list #= ["food", "landmark", "people"]
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


In [ ]:
score = model.evaluate_generator(validation_generator, num_test_images//8, workers=12, verbose=1)
print(model.metrics_names)
print(score)

In [ ]:
model_json = model.to_json()
with open("ca2_normal_model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights('ca2_normal_model.h5')
print("Saved model to disk")

In [ ]:
from tensorflow.keras.models import model_from_json
# load json and create model
json_file = open('ca2_normal_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("ca2_normal_model.h5")
print("Loaded model from disk")

In [ ]:
loaded_model.compile(loss='categorical_crossentropy', optimizer=optmz, metrics=['accuracy'])
score = loaded_model.evaluate_generator(validation_generator, num_test_images//8, workers=12, verbose=1)
print(loaded_model.metrics_names)
print(score)